# Stage 3: Tradeoff

In [2]:
using JuMP
import Combinatorics

In [50]:
function get_nash_primal(score_matrix::Array)
    size = ndims(score_matrix)
    # for primal - P person
    m = Model()

    @variable(m, 1>= p[1:size] >=0)
    @variable(m, t)

    @constraint(m, sum(p) == 1)
    @constraint(m, min_const[k in 1:size], 
        sum(p[i] * score_matrix[i, k] for i=1:size) >= t)
    # Max min (g1, g2, ..., gn)
    @objective(m, Max, t)
    solve(m)
                
    popt = getvalue(p)
#     println("E Optimal Score: ", getobjectivevalue(m))
    return getobjectivevalue(m)        
#     return findmax(popt)
end
;

### Game score function

In [4]:
function get_score_chess_game(player, opponent, is_first)
    if is_first
        x = player - opponent + 35
    else
        x = player - opponent - 35
    end
    return 1/(1 + 10^(-x/400)) 
end
;
function get_match_score(team_player, team_opponent, is_first, players_dict)
    len = length(team_player)
    if is_first
        return sum(get_score_chess_game(players_dict[team_player[i]], 
            players_dict[team_opponent[i]], i % 2 == 1) for i=1:len)
    else
        return sum(get_score_chess_game(players_dict[team_player[i]], 
            players_dict[team_opponent[i]], i % 2 == 0) for i=1:len)
    end
end
;

### Generate score matrices for all players
Suppose that we have 10 players, choose 4 of them to establish our team, 6 other players become our opponents

In [5]:
function get_score_matrix(list1::Array, list2::Array, players_dict::Array)
    # assume that team 1 plays firstly
    dim1 = length(list1)
    dim2 = length(list2)
    scores = Matrix(dim1, dim2)
    for i in 1:dim1
        for j in 1:dim2
            scores[i, j] = get_match_score(list1[i], list2[j], true, players_dict)
        end
    end
    return scores
end

get_score_matrix (generic function with 1 method)

In [6]:
function get_combinations(list_players::Array, nchosen::Int)
    return collect(combinations(list_players, nchosen))
end
function get_permutations(list_players::Array)
    return collect(permutations(list_players))
end

get_permutations (generic function with 1 method)

In [7]:
function get_difference(list_main::Array, list_1::Array)
    lst = Array[]
    for i in list_main
        if !(i in list_1)
            push!(lst, i)
        end
    end
    return lst
end

get_difference (generic function with 1 method)

In [14]:
function get_best_rest(list_main::Array, list_1::Array, num::Int)
    lst = []
    for i in 1:length(list_main)
        if !(list_main[i] in list_1)
            push!(lst, list_main[i])
            if length(lst) == num
                break
            end
        end
    end
    return lst
end

get_best_rest (generic function with 1 method)

### Nash equilibrium

In [9]:
function get_optimal_score(s::Array, list_players::Array)
    team_host = []
    list_players_rest = []
    for i in 1:length(s)
        if s[i] == 1
            push!(team_host, list_players[i])
        else
            push!(list_players_rest, list_players[i])
        end
    end
    coms_rest = get_combinations(list_players_rest, k_chosen)
    list_host = get_permutations(team_host)
    list_guest = Array[]
    for team_guest in coms_rest
        perms = get_permutations(team_guest)
        append!(list_guest, perms)
    end
    scores_matrix = get_score_matrix(list_host, list_guest)
    return get_nash_primal(scores_matrix)
end

get_optimal_score (generic function with 1 method)

In [40]:
function get_budget(team, price::Array)
    total = 0
    for i in 1:length(team)
        total = total + price[team[i]]
    end
    return total
end

get_budget (generic function with 1 method)

In [1]:
# price = [1000, 900, 850, 700, 690, 650, 600, 580, 570, 560]
# elo = [2838, 2822, 2817, 2772, 2771, 2761, 2750, 2747, 2745]
price = [1000, 900, 580, 570, 560] 
possible_budget = []
elo = [2838, 2822, 2750, 2747, 2745]
nplayers = 5
k_chosen = 2
list_players = [1:nplayers;]
;

### Score matrix

In [69]:
set_coms = get_combinations(list_players, k_chosen)
superM = []
for k in 1:length(set_coms)
    team_host = set_coms[k]
    team_guest = get_best_rest(list_players, team_host, k_chosen)
    list_host = get_permutations(team_host)
    list_guest = get_permutations(team_guest)
    matrix_ij = get_score_matrix(list_host, list_guest, elo)
    push!(superM, matrix_ij)
end

In [70]:
budgets = []
for k in 1:length(set_coms)
    push!(budgets, get_budget(set_coms[k], price))
end

Brute force for discrete-state system  
Complexity is O(n!k!/(n-k)!) = O(n^k * k!)   
Typically, k is small (less than 20)   

There are 2 cases: 
- Opponents are too rich, choose the best combination of the rest players  
- Opponent cares about budget => add budget as a score in Nash equilibrium  

Each team has a limited budget for buying.  

In [71]:
# tradeoff
alpha = 0.0005
com_size = length(set_coms)
nsize = factorial(k_chosen)

lst = []
for k in 1:com_size
    team_host = set_coms[k]
    team_guest = get_best_rest(list_players, team_host, k_chosen)
    list_host = get_permutations(team_host)
    list_guest = get_permutations(team_guest)
    matrix_ij = get_score_matrix(list_host, list_guest, elo)
    
    fighting_score = get_nash_primal(matrix_ij)
    score = fighting_score - alpha * budgets[k]
    push!(lst, score)
end
set_coms[findmax(lst)[2]]

2-element Array{Int64,1}:
 1
 2